In [7]:
 #List of all the available variables
#https://developers.google.com/analytics/devguides/reporting/data/v1/api-schema
import numpy as np
import pandas as pd
import os
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange
from google.analytics.data_v1beta.types import Dimension
from google.analytics.data_v1beta.types import Metric
from google.analytics.data_v1beta.types import RunReportRequest
from google.analytics.data_v1beta.types import OrderBy

# You need a json file linking to this file, in order to read all necessary information in to make the GA4 connection
# Documentation can be found here: https://developers.google.com/analytics/devguides/reporting/data/v1
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'Quickstart-9bd06efe9b97.json'
property_id = '351335983'

client = BetaAnalyticsDataClient()



In [8]:
## Format Report - run_report method
def format_report(request):
    response = client.run_report(request)
    
    # Row index
    row_index_names = [header.name for header in response.dimension_headers]
    row_header = []
    for i in range(len(row_index_names)):
        row_header.append([row.dimension_values[i].value for row in response.rows])

    row_index_named = pd.MultiIndex.from_arrays(np.array(row_header), names = np.array(row_index_names))
    # Row flat data
    metric_names = [header.name for header in response.metric_headers]
    data_values = []
    for i in range(len(metric_names)):
        data_values.append([row.metric_values[i].value for row in response.rows])

    output = pd.DataFrame(data = np.transpose(np.array(data_values, dtype = 'f')), 
                          index = row_index_named, columns = metric_names)
    return output

In [9]:
request = RunReportRequest(
        property='properties/'+property_id,
        dimensions=[Dimension(name="month"), 
                    Dimension(name="sessionMedium")],
        metrics=[Metric(name="averageSessionDuration"), 
                 Metric(name="activeUsers")],
        order_bys = [OrderBy(dimension = {'dimension_name': 'month'}),
                    OrderBy(dimension = {'dimension_name': 'sessionMedium'})],
        date_ranges=[DateRange(start_date="2022-06-01", end_date="today")],
    )



output_df = format_report(request)

In [10]:
output_df.head()

averageSessionDuration  activeUsers
month sessionMedium                                     
01    (none)                     196.912292         10.0
      referral                  1230.669067          1.0
02    (none)                     235.875214          8.0
      referral                   569.430603          1.0